# RFDiffusion Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/erickummelstedt/rfdiffusion_tutorial/blob/main/01_rfdiffusion_original/notebooks/01_introduction.ipynb)

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print('Setting up Colab...')
    get_ipython().system('git clone https://github.com/erickummelstedt/rfdiffusion_tutorial.git')
    get_ipython().magic('cd rfdiffusion_tutorial')
    get_ipython().system('pip install -q biopython matplotlib einops')
    get_ipython().system('pip install -q -e .')
    import torch
    print(f'GPU available: {torch.cuda.is_available()}')
else:
    print('Running locally')